In [1]:
from sys import stdout
from os.path import join, isdir, isfile, exists
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import scipy.integrate as integrate
from random import sample
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
import time

In [2]:
washing_machines = pd.read_hdf('./washing_machines.h5')
#washing_machines = pd.read_hdf('./dish_washers.h5')
#washing_machines = pd.read_hdf('./tumble_dryers.h5')
aggregates = pd.read_hdf('./aggregate.h5')

In [3]:
# ADD TO LIB
def getMinMaxDates(df):
    # find the dates that exist for all measurements
    maxdate = df.index.get_level_values('Unix').max()
    mindate = df.index.get_level_values('Unix').min()
    # for all buildings get the common denominator for min and max dates
    for i in np.unique(df.index.get_level_values('building')):
        mask = df.index.get_level_values('building') == i
        if (df.loc[mask].index.get_level_values('Unix').max()) < maxdate:
            maxdate = df.loc[mask].index.get_level_values('Unix').max()
        if (df.loc[mask].index.get_level_values('Unix').min()) > mindate:
            mindate = df.loc[mask].index.get_level_values('Unix').min()
    return (mindate, maxdate)

(mindate,maxdate) = getMinMaxDates(washing_machines)
print(mindate, maxdate)

2014-06-03 11:30:00 2015-05-10 23:30:00


In [4]:
timedelta = maxdate-mindate
trainset = mindate + timedelta/2

In [ ]:
trainset


In [ ]:
from datetime import date, timedelta

sdate = date(trainset.year, trainset.month, trainset.day)   # start date
edate = date(maxdate.year, maxdate.month, maxdate.day)   # end date

delta = edate - sdate       # as timedelta

dates = []

for i in range(delta.days + 1):
    day = sdate + timedelta(days=i)
    dates.append(day)

print(len(dates))
sum(states[2::20])
from itertools import compress
list(compress(dates,states[2::20]))

In [33]:
def on_times(appliance):
    """
        Parameters
        ----------
        appliance = a dataframe of an appliance witn an 'active' column

        Returns
        A dataframe with a datetime index and a column duration.
        Index is when the appliance started and duration the amount of slots the appliance was on (power>20watt).

    """
    records = len(appliance)
    on_dur = pd.DataFrame(columns=['duration'])
    i = 0
    # TODO(kyrcha): possible optimization with matrix
    while (i < records):
        j = 0
        if appliance['active'].values[i] > 50:
            # to parakatw to evala giati evgaze proeidopoihsh gia nan pou nomizw den isxye kiolas
            # todo: check if nan exist.
            np.warnings.filterwarnings('ignore')
            #αν δεν είναι το τελευταιο στοιχείο του πίνακα γιατί πετούσε έρρορ
            while appliance['active'].values[i + j] > 50:
                if j == 0:
                    on_time = appliance.index.get_level_values('Unix')[i]
                j = j + 1
                if ((i + j) == records):
                    break
            on_dur.loc[on_time] = j
            i = i + j -1
        i = i + 1
    return on_dur

In [30]:
def two_day_types (appliance):
    '''

    Parameters

    appliance

    Returns
    probs = a list of probabilities for each day

    '''
    new_app = appliance
    on_app = on_times(new_app)

    if on_app.empty:
        print('The appliance was never on')
        probs = []
    else:
        on_app['business'] = [is_business_day(x) for x in on_app.index]
        new_app['business'] = [is_business_day(x) for x in new_app.index.get_level_values('Unix')]
        # για όλες τις μέρε του εξεταζόμενου διαστήματος φτιάχνουμε ένα dataframe [φτιάχνω καινούρια dataframe γιατί δν κτλβαίνω τι γίνεται στα παλιά αν τα κάνεις πχ resample]
        final_app = new_app.groupby([pd.Grouper(level='Unix', freq='D')]).mean()
        # για όλες τις μέρες που άνοιξε στο εξεταζόμενο διάστημα φτιάχνουμε ένα dataframe
        final_on_app = on_app.groupby(on_app.index.date).mean()
        #   μετράμε πόσες μέρες από κάθε είδος μέρας υπάρχει σε κάθε ένα από τα δύο dataframe
        days_on = final_on_app['business'].value_counts().to_frame().sort_index()
        days_total = final_app['business'].value_counts().to_frame().sort_index()
        # Η πιθανότητα μιας να είναι ανοιχτή μια καθημερινή ημέρα σε σχέση με το να μην είναι
        each_day_on = []
        each_day_total = []
        probs = []
        name_days = np.array(['NonWorkday', 'Workday'])
        for i in range(0, 2):
            print(i)
            if i in days_on.index:
                each_day_on.append(days_on.loc[i]['business'])
                each_day_total.append(days_total.loc[i]['business'])
                #print("Number of ", name_days[i], "s the appliance was on: ", each_day_on[i], sep='')
                #print('Total number of ', name_days[i], 's ', each_day_total[i], sep='')
                probs.append(each_day_on[i] / each_day_total[i])
                #print('Probability P(', name_days[i], "_on) = ", probs[i], sep='')
            else:
                #print("The appliance didn't turn on on ", name_days[i], 's', sep='')
                each_day_on.append(0)
                each_day_total.append(days_total.loc[i]['business'])
                probs.append(0)
            i = i + 1
    return probs

In [31]:
def prob_time_new(appliance):
    '''

    Parameters
    ----------
    appliance = a dataframe of an appliance witn an 'active' column

    Returns
    a series of diffent start times & how many times they occured
    an array of the same thing

    '''
    #fig = plt.figure()
    #ax = fig.add_subplot(1,1,1)
    #fig.suptitle(name + ' time on', fontsize=16)
    new_app = appliance
    # check  mer
    if new_app.empty:
        print('The appliance was never on')
        ptonosx = []
    else:
        on_app = on_times(new_app)
        if len(on_app) > 1:
            #κάνω μια στηλη μόνο με τις ώρες από το Datatime index
            on_app['time'] = on_app.index.time
            #μου εβγαζε error αν εβαζα κατευθείαν το index στο GMM και γ αυτό έκανα όλο αυτό με τους πίνακες ώστε να φτάσω στο W
            Y = on_app['time'].index.to_list()
            X = []
            for i in range(0, len(Y)):
               X.append(Y[i].hour * 3600 + Y[i].minute * 60 + Y[i].second)
            fullX = X.copy()

            # need to plot these here...
            # do everything with  1 house in mind and plot

            # φτιάχνουμε ένα νέο population όπου αντί για 0-24h, θα απλώνεται σε -6h ως 30h,
            for i in range(0, len(fullX)):
                # προσθέτοντας τα samples που είναι στις ώρες 0-6 στις ώρες 24-30
                if (fullX[i] < 21600):
                    fullX.append(86400 + fullX[i])
                # προσθέτοντας τα samples που είναι στις ώρες 0-6 στις ώρες 24-30
                elif (fullX[i] > 64800):
                    fullX.append(fullX[i] - 86400)
            # μετακινούμε αυτό το νέο population σε 0-36h
            for i in range(0, len(fullX)):
                fullX[i] = fullX[i] + 21600
            fullXX = np.array(fullX)
            fullXX = fullXX.reshape(-1, 1)
            if fullXX.size < 10:
                # return empty array if data are not enough
                return []
            gmm = GaussianMixture(n_components=10, covariance_type='diag', reg_covar=100, n_init=5).fit(fullXX)
            x = np.arange(0, 129600, 1)
            x = x.reshape(-1, 1)
            p = np.exp(gmm.score_samples(x))
            # κρατάμε τις πιθανότητες από τις ώρες που μας ενδιαφέρουν όντως και κανονικοποιούμε
            ptonosx = p[21600:108000] / p[21600:108000].sum()

            #ax.plot(y,probability)
            #ax.hist(fullXX,bins=96,density=True)
            #ax.set_title(str(len(Y)) + ' times it was on', fontsize=12)
            #plt.show()
        else:
            print('The appliance was on 1 time')
            ptonosx = []
    return (ptonosx)

In [32]:
def prob_duration(appliance):
    """

    Parameters
    ----------
    appliance = a dataframe of an appliance
    name = the name of the appliance
    Returns
    a

    """
    idx = pd.IndexSlice
    new_app = appliance
    on_app = on_times(new_app)
    probs = on_app['duration'].value_counts(normalize=True).sort_index().to_frame()
    probs = probs.loc[idx[probs.index<96]] # keep one day's worth of duration...else next day
    #fig = plt.figure()
    #fig.suptitle(name + ' duration', fontsize=16)
    #ax = fig.add_subplot(1, 1, 1)
    #ax.bar(probs.index, probs)
    return probs

In [25]:
def is_business_day(date):
    return len(pd.bdate_range(date, date))

In [29]:
df = washing_machines
mask = df.index.get_level_values('building') == 1
idx = pd.IndexSlice
new_app = df.loc[idx[mask, :, mindate:trainset]]
[is_business_day(x) for x in new_app.index.get_level_values('Unix')]
on_app = on_times(new_app)

[(is_business_day(x),x) for x in on_app.index]

[(0, Timestamp('2014-06-08 11:45:00')),
 (0, Timestamp('2014-06-08 13:15:00')),
 (1, Timestamp('2014-06-11 12:45:00')),
 (1, Timestamp('2014-06-12 08:00:00')),
 (1, Timestamp('2014-06-12 11:00:00')),
 (1, Timestamp('2014-06-17 09:30:00')),
 (1, Timestamp('2014-06-17 11:15:00')),
 (1, Timestamp('2014-06-18 10:00:00')),
 (1, Timestamp('2014-06-19 13:45:00')),
 (1, Timestamp('2014-06-20 13:45:00')),
 (0, Timestamp('2014-06-21 14:15:00')),
 (0, Timestamp('2014-06-22 09:00:00')),
 (1, Timestamp('2014-06-25 10:15:00')),
 (1, Timestamp('2014-06-26 08:15:00')),
 (1, Timestamp('2014-06-26 10:00:00')),
 (1, Timestamp('2014-06-27 06:30:00')),
 (0, Timestamp('2014-06-29 09:15:00')),
 (1, Timestamp('2014-07-01 09:00:00')),
 (1, Timestamp('2014-07-02 12:15:00')),
 (1, Timestamp('2014-07-03 13:00:00')),
 (0, Timestamp('2014-07-06 08:45:00')),
 (1, Timestamp('2014-07-09 10:15:00')),
 (1, Timestamp('2014-07-10 08:15:00')),
 (1, Timestamp('2014-07-11 09:30:00')),
 (0, Timestamp('2014-07-12 08:15:00')),


In [34]:
# ADD TO LIB
def train(df, start , end):
    prob_times = []
    prob_days = []
    prob_dur = []
    #train the machine
    idx = pd.IndexSlice
    for i in np.unique(df.index.get_level_values('building')):
        mask = df.index.get_level_values('building') == i
        prob_days.append(two_day_types(df.loc[idx[mask, :, start:end]]))
        prob_times.append(prob_time_new(df.loc[idx[mask, :, start:end]]))
        prob_dur.append(prob_duration(df.loc[idx[mask, :, start:end]]))
    return prob_times, prob_days, prob_dur

prob_times, prob_days, prob_dur = train(washing_machines, mindate, trainset)

0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1
0
1


In [35]:
# check the prob days
prob_days

[[0.4166666666666667, 0.5967741935483871],
 [0.4166666666666667, 0.5806451612903226],
 [0.3958333333333333, 0.6048387096774194],
 [0.5416666666666666, 0.3870967741935484],
 [0.75, 0.4112903225806452],
 [0.20833333333333334, 0.18548387096774194],
 [0.6875, 0.7661290322580645],
 [0.4166666666666667, 0.5483870967741935],
 [0.5416666666666666, 0.25806451612903225],
 [0.5625, 0.8467741935483871],
 [0.10416666666666667, 0.27419354838709675],
 [0.7291666666666666, 0.6774193548387096],
 [0.6875, 0.29838709677419356],
 [0.2708333333333333, 0.31451612903225806],
 [0.5208333333333334, 0.43548387096774194],
 [0.4791666666666667, 0.4435483870967742],
 [0.14583333333333334, 0.2903225806451613],
 [0.5208333333333334, 0.49193548387096775],
 [0.08333333333333333, 0.4435483870967742],
 [0.4583333333333333, 0.28225806451612906]]

In [36]:
def slot (prob_days,prob_times,prob_dur,start,end):
    '''

    Parameters
    ----------
    prob_days - a list of 7 probabilities for each day
    prob_times - a list of 86400 probabilities for each second of the day
    prob_dur - probabilities of duration
    start - time of slot start
    end - time of slot end

    Returns
    -------

    '''
    idx = pd.IndexSlice
    slot_weekday = is_business_day(start)
    start_time = start.hour * 4 + start.minute
    end_time = end.hour * 4 + end.minute
    if prob_days != [] and prob_times != []:
        the_day = prob_days[slot_weekday]
        # για όταν συμπεριλάβουμε duration
        X = np.array(prob_dur.index.to_list())
        d_max = X.max()
        prob_times = prob_times.reshape(-1, 1)
        pfunctionx = lambda x: prob_times[int(x)]
        turns_on = integrate.quad(lambda x: pfunctionx(x)[0], start_time * 900, end_time * 900)[0]
        was_on = integrate.quad(lambda x: pfunctionx(start_time*900 - d_max*900 + x)[0]*prob_dur.loc[idx[prob_dur.index > (d_max-x/900)]].sum(), 0, d_max*900)[0]
        probability = the_day * (turns_on + was_on)
    else:
        probability = 0
    return probability

In [12]:
washing_machines_common = washing_machines.loc[pd.IndexSlice[:, :, trainset:maxdate], :]
mask = washing_machines_common.index.get_level_values('Unix').indexer_between_time('17:00', '19:00')
washing_machines_common_ts = washing_machines_common.iloc[mask]
washing_machines_common_ts.index = pd.MultiIndex.from_arrays([
    washing_machines_common_ts.index.get_level_values('building'), 
    washing_machines_common_ts.index.get_level_values('appNo'), 
    washing_machines_common_ts.index.get_level_values('Unix').date, 
    washing_machines_common_ts.index.get_level_values('Unix').time], 
    names=['building','appNo','date','time'])

In [13]:
aggregates_common = aggregates.loc[pd.IndexSlice[:, trainset:maxdate], :]
mask = aggregates_common.index.get_level_values('Unix').indexer_between_time('17:00', '19:00')
aggregates_common_ts = aggregates_common.iloc[mask]
aggregates_common_ts.index = pd.MultiIndex.from_arrays([
    aggregates_common_ts.index.get_level_values('building'), 
    aggregates_common_ts.index.get_level_values('Unix').date, 
    aggregates_common_ts.index.get_level_values('Unix').time], 
    names=['building', 'date','time'])

In [37]:
# for subset
subset = list(range(2,12,2))
#subset = list(range(2,4,2))
# for timeslots
# timeslots_list = [
#      ('17:00:00', '19:00:00'), 
#      ('16:00:00', '19:00:00'), 
#      ('16:00:00', '20:00:00')
#      ]
timeslots_list = [
    ('07:00:00', '09:00:00'), 
    ('07:00:00', '10:00:00'), 
    ('06:00:00', '10:00:00'),
     ]
#timeslots_list = [

#     ('17:00:00', '19:00:00'), 
#     ('16:00:00', '19:00:00'), 
#     ('16:00:00', '20:00:00')
#     ]
timeslots_list = [('17:00:00', '19:00:00')]
# for runs
number_of_runs = 5
#number_of_runs = 1

final_grid_results = pd.DataFrame(columns=['subset', 'timeslot', 'f1_selected_score', 'f1_random_score', 'precision_selected', 'precision_random', 'recall_selected', 'recall_random', 'auc_selected', 'auc_random'])

buildings_list = np.unique(washing_machines.index.get_level_values('building')).tolist()

app_subset = washing_machines

selected_power = 0
aggregated_power = 0
days = 0

# Run the experiment

for time_on, time_off in timeslots_list:
    precalculated_probs = {};
    for number_of_selected_households in subset:
        results = pd.DataFrame(columns=['f1_selected_score', 'f1_random_score', 'precision_selected', 'precision_random', 'recall_selected', 'recall_random', 'auc_selected', 'auc_random'])
        print(f'households: {number_of_selected_households}, {time_on}-{time_off}')
        dict_index = {'subset': number_of_selected_households, 'timeslot': f'{time_on}-{time_off}'}
        start_time = time.time()
        for k in range (0,number_of_runs):
            selected_power = 0
            aggregated_power = 0
            days = 0
            end_REFIT = maxdate.replace(hour=pd.Timestamp(time_off).hour, minute =0)
            day = trainset.replace(hour=pd.Timestamp(time_on).hour, minute =0)
            selected = pd.DataFrame(columns=['day', 'p'])
            random = pd.DataFrame(columns=['day'])
            #onlist = pd.DataFrame(columns=['state', 'day', 'selected', 'random'])
            df_x = []
 #           day2 = day + pd.Timedelta(days=1)
#            day3 = day2 + pd.Timedelta(days=1)
            #iterate over days
            while day < end_REFIT:
                #print(day)
                thestate = pd.DataFrame(columns=['state', 'day', 'selected', 'random', 'p'])
                p = pd.DataFrame(columns=['p'])
                day_end = day.replace(hour=pd.Timestamp(time_off).hour, minute =0)
                #iterate over buildings
                count = 1
                for i in np.unique(app_subset.index.get_level_values('building')):
                    #calculating probabilities for a certain time slot
                    #the id number of buildings as an integer
                    #saving the probabilities - εδώ διορθώθηκε το i+1 ορίζοντας το index ως i
                    aprob = 0
                    if (count-1, is_business_day(day)) in precalculated_probs:
                        aprob = precalculated_probs[(count-1, is_business_day(day))]
                    else:
                        aprob = slot(prob_days[count-1], prob_times[count-1], prob_dur[count-1], day, day_end)
                        precalculated_probs[(count-1, is_business_day(day))] = aprob
                    p = p.append(pd.DataFrame({'p': aprob}, index=[count]))
                    # έβγαζε κάποιο λάθος η pandas και γι αυτό δεν το περνάω κατευθείαν στο onlist
                    a = (app_subset.loc[pd.IndexSlice[app_subset.index.get_level_values('building') == i, :, day:day_end]]['active'] > 50).any()
                    thestate = thestate.append(pd.DataFrame({'state': a, 'day': day, 'selected': False, 'random': False}, index=[count]))
                    count = count + 1
                #choose random buildings to send a message
                prand = np.random.rand(len(buildings_list))
                # randomlist = sample(buildings_list, k=number_of_selected_households)
                randomlist = list(np.argsort(prand)[::-1][:number_of_selected_households])
                #choose first buildings by sorting based on the probabilities
                thelist = p.sort_values(by=['p'], ascending=False).head(number_of_selected_households)
                #saving selected and random buildings for each day
                thestate['selected'].mask(thestate.index.isin(thelist.index), True, inplace=True)
                thestate['random'].mask(thestate.index.isin(randomlist), True, inplace=True)
                thestate['p'] = p
                thestate['prand'] = prand
                #saving  data for each day/building
                #  = pd.concat([random, pd.DataFrame({'day': day}, index=randomlist)])
                #selected = pd.concat([selected, pd.DataFrame({'day': day, 'p': thelist['p']}, index=thelist.index)])
                
                # I have selected random list and thelist.index
                #selected_power = selected_power + washing_machines_common_ts.loc[pd.IndexSlice[thelist.index,:,str(day.date())],:].groupby('time').sum()
                #aggregated_power = aggregated_power + aggregates_common_ts.loc[pd.IndexSlice[:,str(day.date())],:].groupby('time').sum()
                #days = days + 1
                df_x.append(thestate)
                #onlist = pd.concat([onlist, thestate])
                day += pd.Timedelta(days=1)
            onlist = pd.concat(df_x, axis=0)
            #print(selected_power/days)
            #print(aggregated_power/days)
            states = np.array(onlist['state']).tolist()
            selected_list = np.array(onlist['selected']).tolist()
            random_list = np.array(onlist['random']).tolist()
            probs = np.array(onlist['p']).tolist()
            randprobs = np.array(onlist['prand']).tolist()
            f1_selected_score = f1_score(states, selected_list, average='binary')
            #print("F1 selected score ", f1_selected_score)
            f1_random_score = f1_score(states, random_list, average='binary')
            #print("F1 random score ", f1_random_score )
            auc_selected_score = roc_auc_score(states, probs)
            #print("AUC selected score ", auc_selected_score )
            auc_random_score = roc_auc_score(states, randprobs)
            #print("AUC random score ", auc_random_score )
            precision_selected = precision_score(states, selected_list, average='binary')
            #print("Precision score selected", precision_selected)
            precision_random = precision_score(states, random_list, average='binary')
            #print("Precision score random", precision_random)
            recall_selected = recall_score(states, selected_list, average='binary')
            #print("Recall score selected",recall_selected)
            recall_random = recall_score(states, random_list, average='binary')
            #print("Recall score random", recall_random)
            #balanced_selected = balanced_accuracy_score(states, selected_list)
            #print("Balanced accuracy score selected ", balanced_selected)
            #balanced_random = balanced_accuracy_score(states, random_list)
            #print("Balanced accuracy score random ", balanced_random)
            #accuracy_selected = accuracy_score(states, selected_list)
            #print("Accuracy score selected ", accuracy_selected)
            #accuracy_random = accuracy_score(states, random_list)
            #print("Accuracy score random ", accuracy_random)
            results = results.append(
                pd.DataFrame({
                    'f1_selected_score':f1_selected_score, 
                    'f1_random_score':f1_random_score, 
                    'precision_selected': precision_selected,
                    'precision_random': precision_random,
                    'recall_selected': recall_selected,
                    'recall_random': recall_random,
                    'auc_selected':auc_selected_score, 
                    'auc_random': auc_random_score}, index=[k])
            )
        # get the average of recall for random and selected
        # put in in the result list
#        print(results.mean())
        dict_results = results.mean().to_dict()
        final_grid_results = final_grid_results.append({**dict_index, **dict_results}, ignore_index=True)
        print('minutes: ',(time.time() - start_time)/60)

households: 2, 17:00:00-19:00:00
minutes:  17.828949411710102
households: 4, 17:00:00-19:00:00
minutes:  10.24557567834854
households: 6, 17:00:00-19:00:00
minutes:  9.427835067113241
households: 8, 17:00:00-19:00:00
minutes:  12.744789981842041
households: 10, 17:00:00-19:00:00
minutes:  10.110667117436726


In [38]:
final_grid_results

,subset,timeslot,f1_selected_score,f1_random_score,precision_selected,precision_random,recall_selected,recall_random,auc_selected,auc_random
0,2,17:00:00-19:00:00,0.304945,0.095404,0.324561,0.104789,0.287565,0.087565,0.769669,0.506731
1,4,17:00:00-19:00:00,0.379439,0.146152,0.296784,0.116571,0.525907,0.195855,0.769669,0.500083
2,6,17:00:00-19:00:00,0.366856,0.170713,0.252437,0.119057,0.670984,0.301554,0.769669,0.500599
3,8,17:00:00-19:00:00,0.337514,0.180105,0.216374,0.116753,0.766839,0.393782,0.769669,0.486899
4,10,17:00:00-19:00:00,0.317748,0.182269,0.194737,0.112793,0.862694,0.474611,0.769669,0.504704


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

from scipy import interp
from sklearn.metrics import roc_auc_score

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(states, randprobs)
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

In [ ]:
d = { 'real': states, 'probs': randprobs }
roc_df = pd.DataFrame(d)
roc_df.to_csv('demo.csv')

In [ ]:
b = [1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,18,19,20,21]
list(np.argsort(np.random.rand(len(b)))[::-1][:2])

In [ ]:
final_grid_results

In [ ]:
sum(states)

In [ ]:
aggregates_common_ts.loc[pd.IndexSlice[8, '2015-05-09', :],:].groupby(['time']).sum()

In [ ]:
283.014815 + 1699.649254 + 494.225564

In [ ]:
days

In [ ]:
selected_power/170

In [ ]:
aggregated_power/170

In [ ]:
maxdate-trainset

In [ ]:
aggregates.loc[idx[sample(a, k=2),'2014-06-03 17:30:00'],'active'].sum()

In [ ]:
washing_machines

In [ ]:
washing_machines_common = washing_machines.loc[pd.IndexSlice[:, :, trainset:maxdate], :]
mask = washing_machines_common.index.get_level_values('Unix').indexer_between_time('17:00', '19:00')
washing_machines_common_ts = washing_machines_common.iloc[mask]
washing_machines_common_ts.index = pd.MultiIndex.from_arrays([
    washing_machines_common_ts.index.get_level_values('building'), 
    washing_machines_common_ts.index.get_level_values('appNo'), 
    washing_machines_common_ts.index.get_level_values('Unix').date, 
    washing_machines_common_ts.index.get_level_values('Unix').time], 
    names=['building','appNo','date','time'])
a = 0
a = a + washing_machines_common_ts.loc[pd.IndexSlice[[1,2],:,'2014-12-22'],:].groupby('time').sum()
a = a + washing_machines_common_ts.loc[pd.IndexSlice[[1,2],:,'2014-12-23'],:].groupby('time').sum()
a